# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv
from geoviews import opts, tile_sources as gvts
gv.extension('bokeh')


# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,-2.28,94,100,0.92,GS,1694294767
1,1,fale old settlement,-9.3852,-171.2468,27.86,75,68,3.22,TK,1694294767
2,2,stanley,54.8680,-1.6985,20.44,90,20,2.06,GB,1694294767
3,3,edinburgh of the seven seas,-37.0676,-12.3116,9.70,57,88,12.16,SH,1694294768
4,4,margaret river,-33.9500,115.0667,14.73,68,100,4.96,AU,1694294768


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

#set the default visualisation options
opts.defaults(
    opts.WMTS(width=900, height=500, xaxis=None, yaxis=None, level='annotation'))

#configure the map plot
first_map_plot = city_data_df.hvplot.points(
    x="Lng", 
    y="Lat",
    size = "Humidity",
    color = "City",
    geo = True,
    alpha= 0.6,
    tiles = "OSM",
    scale = 1,
    
    
)


#Display the map
first_map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
narrow_city_df = city_data_df.loc[
    (city_data_df['Cloudiness'] == 0) & 
    (city_data_df['Wind Speed'] < 4.5) & 
    (city_data_df['Max Temp'] < 27) & 
    (city_data_df['Max Temp'] > 21) 
]

# Drop any rows with null values
narrow_city_df= narrow_city_df.dropna()

# Display sample data
narrow_city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
34,34,toliara,-23.3500,43.6667,21.67,86,0,0.29,MG,1694294771
99,99,balkanabat,39.5108,54.3671,26.63,33,0,2.96,TM,1694294777
123,123,penne,42.4538,13.9299,21.48,82,0,1.78,IT,1694294780
129,129,ghanzi,-21.5667,21.7833,21.63,17,0,3.99,BW,1694294491
180,180,gilgit,35.9221,74.3087,24.19,44,0,0.45,PK,1694294785


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrow_city_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

C:\Users\Beaut\AppData\Local\Temp\ipykernel_25156\620281109.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
34,toliara,MG,-23.3500,43.6667,86,
99,balkanabat,TM,39.5108,54.3671,33,
123,penne,IT,42.4538,13.9299,82,
129,ghanzi,BW,-21.5667,21.7833,17,
180,gilgit,PK,35.9221,74.3087,44,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "hotel",
    "radius": radius,
    "limit": 1,
    "apiKey": geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():

    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"] 
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)

    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
toliara - nearest hotel: No hotel found
balkanabat - nearest hotel: No hotel found
penne - nearest hotel: No hotel found
ghanzi - nearest hotel: No hotel found
gilgit - nearest hotel: No hotel found
nahrin - nearest hotel: No hotel found
farkhar - nearest hotel: No hotel found
langhirano - nearest hotel: No hotel found
porto torres - nearest hotel: No hotel found
zhaobaoshan - nearest hotel: No hotel found
morondava - nearest hotel: No hotel found
kolonia - nearest hotel: No hotel found
buritizeiro - nearest hotel: No hotel found
shiraz - nearest hotel: No hotel found
colonia - nearest hotel: No hotel found
rothenburg ob der tauber - nearest hotel: No hotel found
florence - nearest hotel: No hotel found
kasane - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
34,toliara,MG,-23.3500,43.6667,86,No hotel found
99,balkanabat,TM,39.5108,54.3671,33,No hotel found
123,penne,IT,42.4538,13.9299,82,No hotel found
129,ghanzi,BW,-21.5667,21.7833,17,No hotel found
180,gilgit,PK,35.9221,74.3087,44,No hotel found
352,nahrin,AF,36.0649,69.1334,22,No hotel found
382,farkhar,AF,36.5729,69.8578,26,No hotel found
395,langhirano,IT,44.6127,10.2656,47,No hotel found
404,porto torres,IT,40.8350,8.3972,76,No hotel found
437,zhaobaoshan,CN,29.9695,121.6875,86,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
%%capture --no-display

#set the default visualisation options
opts.defaults(
    opts.WMTS(width=700, height=500, xaxis=None, yaxis=None, level='annotation'))

#configure the map plot
second_map_plot = hotel_df.hvplot.points(
    x="Lng", 
    y="Lat",
    hover_cols=['City', 'Lat', 'Lng', 'Humidity','Hotel Name','Country'],
    size = "Humidity",
    color = "City",
    geo = True,
    alpha= 0.6,
    tiles = "OSM",
    scale = 1,
    tools=['hover']
    
)


#Display the map
second_map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)